In [1]:
import load
from tree import tmtree
import numpy as np
import utils.sampling as smp
import tree.halomodule as hmo 

# Load simulation data and halo (tree later)

wdir = '/home/hoseung/Work/data/05427/'
nout = 187
info = load.info.Info(nout=nout, base=wdir, load=True)
#hgal = hmo.Halo(base=wdir, nout=nout, halofinder='HM', info=info, load=True, is_gal=True)

In [18]:
from load.utils import skip_fortran, read_fortran
fn = "/home/hoseung/Work/data/05427/GalaxyMaker/gal/tree_bricks187"
f = open(fn, "rb")

nbodies = read_fortran(f, np.dtype('i4'), 1)
massp = read_fortran(f, np.dtype('f4'), 1)
aexp = read_fortran(f, np.dtype('f4'), 1)
omegat = read_fortran(f, np.dtype('f4'), 1)
age = read_fortran(f, np.dtype('f4'), 1)
halnum, subnum = read_fortran(f, np.dtype('i4'), 2)
dtype_gal = [('np', '<i4'), ('galid', '<i4'), ('hhost', '<i4', (5,)),
            ('ang', '<f4', (3,)), ('m', '<f4'), ('mvir', '<f4'),
            ('r', '<f4', (4,)), ('energy', '<f8', (3,)),
            ('rvir', '<f4'), ('p', '<f4', (3,)),
            ('v', '<f4', (3,)), ('sp', '<f4')]

dtype_profile = [('rho_0', '<f4'), ('r_c', '<f4')]

tothal = halnum + subnum
data = np.recarray(tothal, dtype=dtype_gal)
profile = np.recarray(tothal, dtype=dtype_profile)

for i in range(tothal):
    nph = read_fortran(f, np.dtype('i4'), 1)
    data['np'][i] = nph
    pids = read_fortran(f, np.dtype('i4'), nph) # id list. 
    data['galid'][i] = read_fortran(f, np.dtype('i4'), 1)
    read_fortran(f, np.dtype('i4'), 1) #timestep
    data['hhost'][i][0:5] = read_fortran(f, np.dtype('i4'), 5)
    data['m'][i] = read_fortran(f, np.dtype('f4'), 1)
    data['p'][i][0:3] = read_fortran(f, np.dtype('f4'), 3)
    data['v'][i][0:3] = read_fortran(f, np.dtype('f4'), 3)
    data['ang'][i][0:3] = read_fortran(f, np.dtype('f4'), 3)
    data['r'][i][0:4] = read_fortran(f, np.dtype('f4'), 4)
    data['energy'][i][0:3] = read_fortran(f, np.dtype('f4'), 3)#energies
    data['sp'][i] = read_fortran(f, np.dtype('f4'), 1)
    sig, sigb, mb = read_fortran(f, np.dtype('f4'), 3)
    virials = read_fortran(f, np.dtype('f4'), 4)
    data['rvir'][i], data['mvir'][i] = virials[0:2]
    profile['rho_0'][i], profile['r_c'] = read_fortran(f, np.dtype('f4'), 2)
    nbin = read_fortran(f, np.dtype('i4'), 1)
    rr = read_fortran(f, np.dtype('f4'), nbin)
    rho = read_fortran(f, np.dtype('f4'), nbin) # wasted..
    

In [19]:
data['np']

array([2054426,   66958,  101730,   56215,   39669,   78748,  181272,
        114847,   37463,   56315,   90760,   33688,   24727,   50856,
        173276,   24670,     786,   53288,  126157,   75062,   43574,
        102832,   80432,   27985,     555,   28261,     660,    1233,
         35361,   14730,   12686,   26205,   25646,   23838,   30615,
         30649,   23631,    2198,   10522,   64090,    9521,   18017,
         21062,   18874,   30971,    3114,     495,   26437,   11666,
          3772,   13557,    1800,   90648,   10814,   16487,   20420,
         25218,   19009,   14044,     755,   24171,    8212,   21475,
         17166,    3390,   24949,   15885,   12661,     782,    8389,
          4121,   27548,   19938,    9778,   14911,   42373,    4809,
         26459,   14871,     300,   10749,    2533,    7550,   52034,
          8759,   22764,   14540,    9313,   94964,   66798,   15597,
         40507,   24926,   16731,   25859,    5468,   17997,   17664,
          2270,   12

In [11]:
    data['np'][i] = read_fortran(f, np.dtype('i4'), 1)
    pids = read_fortran(f, np.dtype('i4'), nph) # id list. 
    #pids = read_fortran(f, np.dtype('i4'), nph) # id list. 
    galid = read_fortran(f, np.dtype('i4'), 1)
    read_fortran(f, np.dtype('i4'), 1) #timestep
    level, host, sub, nsub, nextsub = read_fortran(f, np.dtype('i4'), 5)
    mass = read_fortran(f, np.dtype('f4'), 1)
    xx, yy, zz = read_fortran(f, np.dtype('f4'), 3)
    vx, vy, vz = read_fortran(f, np.dtype('f4'), 3)
    ax, ay, az = read_fortran(f, np.dtype('f4'), 3)
    r = read_fortran(f, np.dtype('f4'), 4)[0]
    read_fortran(f, np.dtype('f4'), 3)#energies
    sp = read_fortran(f, np.dtype('f4'), 1)
    sig, sigb, mb = read_fortran(f, np.dtype('f4'), 3)
    rvir, mvir = read_fortran(f, np.dtype('f4'), 4)[0:2]
    rho_0,rho_c = read_fortran(f, np.dtype('f4'), 2)
    nbin = read_fortran(f, np.dtype('i4'), 1)
    rr = read_fortran(f, np.dtype('f4'), nbin)
    rho = read_fortran(f, np.dtype('f4'), nbin)

In [12]:
nph

array([1483], dtype=int32)

In [9]:
rho

array([  1.03994621e+02,   9.18323746e+01,   7.38007355e+01,
         6.07760811e+01,   4.81730194e+01,   3.95351486e+01,
         3.10097618e+01,   2.44032860e+01,   2.07155056e+01,
         1.62880344e+01,   1.43288059e+01,   1.19010649e+01,
         9.14089108e+00,   7.74707460e+00,   6.63719654e+00,
         5.51804352e+00,   4.43158913e+00,   3.64656496e+00,
         3.03424120e+00,   2.45820642e+00,   2.09689832e+00,
         1.70392883e+00,   1.43452179e+00,   1.15032744e+00,
         1.03705227e+00,   7.41494715e-01,   6.77280664e-01,
         5.18361688e-01,   5.02765119e-01,   3.83069575e-01,
         2.88174391e-01,   1.99304283e-01,   1.59082696e-01,
         1.25671938e-01,   6.85076639e-02,   3.53694893e-02,
         1.21413404e-02,   1.18175708e-02,   7.67374784e-03,
         1.86986884e-03,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,

In [4]:
print(pids)

nph = read_fortran(f, np.dtype('i4'), 1)
print(f.tell())
#print(np.fromfile(f, np.dtype(np.int32), 1))
#np.fromfile(f, np.dtype(np.int16), 1)
#pids = np.fromfile(f, np.dtype(np.int32), nph)
pids = read_fortran(f, np.dtype('i4'), nph) # id list. 
print(pids)

pids2 = read_fortran(f, np.dtype('i4'), nph) # id list. 
print(f.tell())
galid = read_fortran(f, np.dtype('i4'), 1)
print(pids2)
print(galid)
print(f.tell())

[       2        4        7 ..., 11209993 11209994 11209995]
8218052
[1056964608 1069547520 1075838976 1080033280 1083179008 1085276160
 1087373312 1089470464 1091043328 1092091904 1093140480 1094189056
 1095237632 1096286208 1097334784 1098383360 1099169792 1099694080
 1100218368 1100742656 1101266944 1101791232 1102315520 1102839808
 1103364096 1103888384 1104412672 1104936960 1105461248 1105985536
 1106509824 1107034112 1107427328 1107689472 1107951616 1108213760
 1108475904 1108738048 1109000192 1109262336 1109524480 1109786624
 1110048768 1110310912 1110573056 1110835200 1111097344 1111359488
 1111621632 1111883776 1112145920 1112408064 1112670208 1112932352
 1113194496 1113456640 1113718784 1113980928 1114243072 1114505216
 1114767360 1115029504 1115291648 1115553793 1115750400 1115881472
 1116012544 1116143616 1116274688 1116405760 1116536832 1116667904
 1116798976 1116930048 1117061120 1117192192 1117323264 1117454336
 1117585408 1117716480 1117847552 1117978624 1118109696 1118

In [21]:
print(f.tell())

OSError: Unexpected FORTRAN block length 0!=400

In [12]:
print("number of galaxies:", len(hgal.data))

number of galaxies: 475


In [ ]:
tt = tmtree.load(work_dir=wdir, filename="halo/TMtree.fits")



# plot individual galaxies.